In [ ]:
from neo4j import GraphDatabase


# Connect to DB
URI = 'neo4j://localhost:7687/'
AUTH = ('neo4j', '123neo4j')

driver = GraphDatabase.driver(URI, auth=AUTH)

driver.verify_connectivity()


# Remove old data
driver.execute_query('MATCH (a) -[r] -> () DELETE a, r')
driver.execute_query('MATCH (a) DELETE a')


# Import nodes
driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MERGE (i:Item {itemID: row.itemID, name: row.name, price: row.price});
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/items.csv'
)

driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MERGE (c:Customer {customerID: row.customerID, name: row.name});
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/customers.csv'
)

driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MERGE (o:Order {orderID: row.orderID});
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/bought.csv'
)


# Import arrows
driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MATCH (c:Customer {customerID: row.customerID})
    MATCH (o:Order {orderID: row.orderID})
    CREATE (c)-[:BOUGHT]->(o);
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/bought.csv'
)

driver.execute_query(
    '''
    LOAD CSV WITH HEADERS FROM $url AS row
    MATCH (o:Order {orderID: row.orderID})
    MATCH (i:Item {itemID: row.itemID})
    CREATE (o)-[:CONTAINS]->(i);
    ''',
    url='https://raw.githubusercontent.com/Artem-Litkovskyi/kpi-db/master/lab5/import/contains.csv'
)

# Товари (items)